# Linear Algebra 2

- matrix types
- eigenvalues and singular values
- sparse matrices
- Krylov subspace methods
- example: the power method

### Matrix

If an algorithms knows something about the structure of a matrix (ie if it is diagonal, hermitian, symmetric, upper trianglar, tri-diagonal, sparse etc.) there is likely a more efficient algorithm than one that works for a general matrix. However, checking each entry to confirm this structure could erase any speedup. 

In [4]:
A = randn(4, 4)

4×4 Matrix{Float64}:
  0.320766  -0.350786  -0.964226  2.32074
  2.03683    0.469161  -0.84902   0.0730647
  1.7249    -0.332978   0.853162  1.45533
 -0.176783   0.35677    1.91591   0.400834

In [5]:
B = (A + A') / 2

4×4 Matrix{Float64}:
 0.320766   0.84302    0.380334  1.07198
 0.84302    0.469161  -0.590999  0.214917
 0.380334  -0.590999   0.853162  1.68562
 1.07198    0.214917   1.68562   0.400834

In [6]:
using LinearAlgebra

In [7]:
Symmetric(A)

4×4 Symmetric{Float64, Matrix{Float64}}:
  0.320766  -0.350786   -0.964226  2.32074
 -0.350786   0.469161   -0.84902   0.0730647
 -0.964226  -0.84902     0.853162  1.45533
  2.32074    0.0730647   1.45533   0.400834

In [8]:
Diagonal(A)

4×4 Diagonal{Float64, Vector{Float64}}:
 0.320766   ⋅         ⋅         ⋅ 
  ⋅        0.469161   ⋅         ⋅ 
  ⋅         ⋅        0.853162   ⋅ 
  ⋅         ⋅         ⋅        0.400834

In [9]:
b = rand(4);

In [10]:
Diagonal(A) \ b

4-element Vector{Float64}:
 0.3673347054935859
 0.9691407730732474
 0.09975584150165803
 0.25754850970749393

## Eigenvalues and eignevectors

In [11]:
eigen(A)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
4-element Vector{ComplexF64}:
 -0.7319926083553863 - 1.8904643533608572im
 -0.7319926083553863 + 1.8904643533608572im
   0.817705199300573 + 0.0im
  2.6902026444536293 + 0.0im
vectors:
4×4 Matrix{ComplexF64}:
   -0.616704-0.0im       …   0.101197+0.0im     0.286154+0.0im
    0.172349-0.542222im       0.97771+0.0im  -0.00287729+0.0im
 -0.00208466-0.360046im     -0.149358+0.0im      0.74579+0.0im
    0.304941+0.270813im      0.107397+0.0im     0.601585+0.0im

In [12]:
eigen(Symmetric(A))

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
4-element Vector{Float64}:
 -2.892207209878333
  0.2981996270755185
  1.7859940850250993
  2.851936124821154
vectors:
4×4 Matrix{Float64}:
 -0.608688   0.224301   0.507441   -0.567179
 -0.189413  -0.891966   0.368735    0.180428
 -0.444243  -0.294607  -0.77698    -0.334897
  0.629499  -0.259408   0.0532925  -0.730478

Note: it is a convention in Julia that variables and function names start with a lowercase, and for types (and constructors) to start with uppercase.

eg. eigen called on Symmetric matrix type knows ahead of time that the eigen vals will be real valued, uses some optimisations

In [14]:
eigen(Diagonal(A))

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
4-element Vector{Float64}:
 0.32076620402700384
 0.46916092850572894
 0.8531615524771219
 0.40083394203357803
vectors:
4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

in this case for diagonal matrix, eigen knows this is trivial doesnt really need to do any calculation

## Singular value decomposition

In [15]:
sd = svd(A)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
4×4 Matrix{Float64}:
 -0.622319   -0.117735   0.714758   0.296612
 -0.464122   -0.369946  -0.657591   0.46401
 -0.627884    0.460598  -0.228309  -0.584368
  0.0554168   0.798203  -0.067605   0.596012
singular values:
4-element Vector{Float64}:
 3.315815170264088
 2.4529527937703257
 2.0631316553136685
 0.5387435884069448
Vt factor:
4×4 Matrix{Float64}:
 -0.674883    0.0691823     0.170272  -0.714671
 -0.0562202  -0.000350118   0.957973   0.281296
 -0.723165   -0.245908     -0.220631   0.606534
 -0.135663    0.966821     -0.067954   0.205511

In [16]:
C = randn(4, 2)
sd = svd(C)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
4×2 Matrix{Float64}:
 -0.406711  -0.623586
  0.756865  -0.309966
  0.476535   0.226173
 -0.186161   0.681108
singular values:
2-element Vector{Float64}:
 3.237936868285392
 1.4000374557173392
Vt factor:
2×2 Matrix{Float64}:
 -0.844145   0.536115
 -0.536115  -0.844145

In [18]:
sd.U * Diagonal(sd.S) * sd.Vt - C

4×2 Matrix{Float64}:
  0.0          -1.52656e-16
  0.0           2.22045e-16
  2.22045e-16   0.0
 -5.55112e-17   0.0

## Sparse matrices

matrices with only a few non-zero entries. we efficiently represent a sparse matrix by using a vector that contains the index and value of the non-zero values (called coordinate format). order by index

use binary search to find a particular entry of the sparse matrix. more expensive than just searching the matrix. 
also more expensive to insert new values into a sparse matrices. 
these methods not implemented in julia for this reason

### storage formats:

- coordinate format (all nonzero entries and their index)
- compressed sparse column (vector for each column containing row and value for nonzero vals)

Inefficient
- find particular element
- add element

Efficient:
- matrix-vector multiplication

In [19]:
using SparseArrays

In [30]:
A = sprand(4, 4, 0.4)

4×4 SparseMatrixCSC{Float64, Int64} with 11 stored entries:
 0.827174   ⋅        0.18461   0.551448
  ⋅         ⋅        0.737777  0.552807
  ⋅        0.304507  0.90205   0.513255
  ⋅        0.594817  0.859059  0.598569

In [31]:
b = randn(4);

In [32]:
A * b

4-element Vector{Float64}:
  1.5175686362413168
  0.23277995749163027
 -0.6860173890904795
 -0.7246254160340908

In [33]:
inv(A)

LoadError: The inverse of a sparse matrix can often be dense and can cause the computer to run out of memory. If you are sure you have enough memory, please either convert your matrix to a dense matrix, e.g. by calling `Matrix` or if `A` can be factorized, use `\` on the dense identity matrix, e.g. `A \ Matrix{eltype(A)}(I, size(A)...)` restrictions of `\` on sparse lhs applies. Alternatively, `A\b` is generally preferable to `inv(A)*b`

In [34]:
A \ b

4-element Vector{Float64}:
 -10.019115637518475
   9.947234739800537
 -17.814430435498288
  21.200300398577326

## Iterative methods

iterative alg for finding eigenvals/eigenvecs of a matrix. may only have blackbox access to A. Power method:

take random vec x, then calc $Ax,  A^{2}x,  \dots , A^{n}x$

can assume $x = \sum_{n} c_{n} v_{n}$ where v are the eigenvectors of A. 

$A^{n}x = \sum_{n} \lambda^{n} c_n v_n$

taking n large, the resulting vector approaches the direction of the eigenvector with the largest eigenvalue. can then find this eigenvalue

In [41]:
# Power method
function power_method(A)
    #for simplicity here we assume a symmetric matrix, hence real valued eigenvals. could generalise later
    x = randn(size(A, 1))
    for iter in 1:40
        xnew = A * x
        eval  = xnew[1] / x[1]
        err = norm(xnew - eval * x)
        println("iter: $iter eval: $eval error: $err evec[1]: $(x[1])")
        x = xnew
        x = x / norm(x)
    end
    return x
end

power_method (generic function with 1 method)

In [42]:
power_method(Symmetric(A))

iter: 1 eval: -2.8070278260626282 error: 8.145602502364389 evec[1]: 0.208709565575456
iter: 2 eval: 3.0753405961644966 error: 1.210779917091808 evec[1]: -0.1873812268877393
iter: 3 eval: 2.205481939460879 error: 0.29379457887676386 evec[1]: -0.30263831599365915
iter: 4 eval: 2.0503151155541373 error: 0.12007975923203011 evec[1]: -0.34512877306558337
iter: 5 eval: 1.9838038436500312 error: 0.047413073348957176 evec[1]: -0.364815008833202
iter: 6 eval: 1.9597814135796094 error: 0.020680317969296223 evec[1]: -0.3729068838913814
iter: 7 eval: 1.9489358414469267 error: 0.00876208881873307 evec[1]: -0.3765228872899485
iter: 8 eval: 1.9444810643500448 error: 0.0038302700849714745 evec[1]: -0.3780625460805445
iter: 9 eval: 1.9425035698304378 error: 0.001651694988618881 evec[1]: -0.378739392671292
iter: 10 eval: 1.941660433436766 error: 0.0007200832737923605 evec[1]: -0.379031323651962
iter: 11 eval: 1.941290537968556 error: 0.0003121387574442178 evec[1]: -0.3791587850426776
iter: 12 eval: 1.94

4-element Vector{Float64}:
 -0.3792565510398262
 -0.3957753564218653
 -0.6237128105114759
 -0.5572330443776615

## Krylov iterative methods

iterative methods for solving linear systems with sparse matrices (?). power method is a simple example

eg. solve $Ax = b$ by an iterative method

starting value x

$x_n \rightarrow x_{n+1}$ 

want $x_n$ to approx soln. $A x_n = b = 0$

$x_{n+1} - x_n = \alpha (Ax_n - b)$

note: don't use this method, just a simple but not great example of a krylov subspace method